# Checking Results

In [40]:
import pandas as pd
from pyhere import here

path = {
    "ddpub": here("data", "derived", "public"),
    "og_out": here("data", "raw", "public", "spielman", "output")
}

In [41]:
def check_it(file, rounder = False):
    '''
    Given a file name, this function finds the corresponding file provided by Spielman et al and the file produced
    by our code and returns the number of matches for each column.    
    '''
    global rpl
    global og
    global test
    
    rpl = pd.read_csv( here(path["ddpub"], file) )
    og = pd.read_csv( here(path["og_out"], file) )
    og = og.rename(columns = {"Geo_FIPS": "GEOID"})
    
    if "sovi" in rpl.columns:
        rpl["sovi"] = rpl["sovi"].round(2)
        og["sovi"] = og["sovi"].round(2)
    
    if "Unnamed: 0" in rpl.columns:
        rpl.index = rpl["Unnamed: 0"]
        rpl = rpl.drop(columns = ["Unnamed: 0"])
        
    if "Unnamed: 0" in og.columns:
        og.index = og["Unnamed: 0"]
        og = og.drop(columns = ["Unnamed: 0"])
        
    if og.columns[0] != rpl.columns[0]:
        og = og.reindex(sorted(og.columns), axis = 1)
        rpl = rpl.reindex(sorted(rpl.columns), axis = 1)
    
    if rounder != False:
        og = og.round(rounder)
        rpl = rpl.round(rounder)
        
    test = rpl.eq(og)
    
    if test.sum().eq(len(rpl)).sum() == len(test.sum()):
        return print("All values match!")
    else:
        return test.sum()

### US SoVI Scores & Rankings

In [3]:
check_it('US_Sovi_Score.csv')

GEOID    3143
sovi     3143
rank     3141
dtype: int64

In [4]:
merged = og.merge(rpl, how = "inner", on = "GEOID")
merged.loc[~test["rank"]]

,GEOID,sovi_x,rank_x,sovi_y,rank_y
1150,g22075,-4.53,2984.0,-4.53,2983.0
3120,g56001,-4.53,2983.0,-4.53,2984.0


It looks like two SoVI values were very close together, switching ranks.

### FEMA Region SoVI Scores & Rankings

In [ ]:
# add some code to make NaN equal in the test function
# also write some code to say howmany records discarding bc SoVI Null
# If sovis NaN, then discard

In [42]:
check_it('FEMA_Region_Sovi_Score.csv')

GEOID          3143
sovi           3109
rank           3109
fema_region    3109
dtype: int64

In [43]:
merged = og.merge(rpl, how = "inner", on = "GEOID")
merged.loc[~test["rank"] | ~test["sovi"] | ~test["fema_region"]]#.head()

,GEOID,sovi_x,rank_x,fema_region_x,sovi_y,rank_y,fema_region_y
67,g02013,NaN,NaN,NaN,NaN,NaN,NaN
68,g02016,NaN,NaN,NaN,NaN,NaN,NaN
69,g02020,NaN,NaN,NaN,NaN,NaN,NaN
70,g02050,NaN,NaN,NaN,NaN,NaN,NaN
71,g02060,NaN,NaN,NaN,NaN,NaN,NaN
72,g02068,NaN,NaN,NaN,NaN,NaN,NaN
73,g02070,NaN,NaN,NaN,NaN,NaN,NaN
74,g02090,NaN,NaN,NaN,NaN,NaN,NaN
75,g02100,NaN,NaN,NaN,NaN,NaN,NaN
76,g02105,NaN,NaN,NaN,NaN,NaN,NaN


Why is there missing data in both my data and theirs?
These rows represent all counties and county equivalents in:
- 15: Hawaii
- 02: Alaska

Checking their code, when they define the states in FEMA region IX, they do not include HI, and when they define the states in FEMA region X, they do not include AK.

### State SoVI Scores & Rankings

In [7]:
check_it('State_Sovi_Score.csv')

All values match!


### County in State Rank

In [8]:
check_it("County_in_State_Rank.csv")

All values match!


### Variable Contributions

In [66]:
check_it("variable_contributions.csv", rounder = 3)

All values match!


In [67]:
og

,FEMA_1,FEMA_10,FEMA_2,FEMA_3,FEMA_4,FEMA_5,FEMA_6,FEMA_7,FEMA_8,FEMA_9,...,g06,g13,g16,g17,g23g33g25,g29,g36,g46,g48,g51
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
MEDAGE_ACS,0.254,0.082,0.170,0.185,0.157,0.244,0.088,0.023,0.468,0.285,...,0.066,0.260,0.066,-0.037,0.376,0.158,0.312,0.210,0.125,0.113
BLACK_ACS,0.094,-0.238,0.062,0.187,-0.122,-0.060,-0.568,0.060,0.340,-0.526,...,-0.311,0.092,0.390,0.016,0.054,-0.023,0.042,0.462,0.438,-0.109
QNATAM_ACS,0.542,-0.046,0.574,-0.232,0.807,0.642,0.448,0.583,-0.076,0.399,...,-0.303,-0.533,0.283,0.532,0.437,0.442,-0.548,-0.067,-0.063,-0.301
QASIAN_ACS,0.210,-0.242,-0.123,0.072,0.073,0.021,0.120,-0.150,0.231,-0.000,...,-0.097,0.217,0.334,0.327,0.122,0.161,-0.165,0.513,0.228,0.199
QHISP_ACS,0.219,0.334,-0.119,0.634,0.461,0.389,0.377,0.396,0.516,0.275,...,0.271,0.369,0.253,-0.193,0.328,0.484,0.015,0.498,-0.027,0.560
QAGEDEP_ACS,0.500,0.291,0.171,0.379,0.375,0.617,0.562,0.329,0.512,0.381,...,0.234,0.148,0.116,0.053,0.561,0.435,0.598,0.139,0.183,0.269
QPUNIT_ACS,0.169,-0.072,-0.553,0.131,-0.407,0.058,-0.017,0.134,-0.187,0.136,...,-0.190,-0.370,-0.198,-0.430,0.363,-0.455,-0.355,-0.144,-0.372,-0.044
PRENTER_ACS,0.061,0.367,0.089,-0.130,0.304,-0.084,0.227,0.066,-0.167,-0.079,...,0.400,-0.114,0.457,0.191,-0.032,0.164,0.167,-0.040,0.233,0.102
QNRRES_ACS,0.461,0.330,-0.164,0.514,0.524,0.220,0.289,0.426,-0.290,0.859,...,0.661,0.441,0.166,-0.145,0.355,0.284,0.541,-0.281,0.330,0.161


In [68]:
rpl

,FEMA_1,FEMA_10,FEMA_2,FEMA_3,FEMA_4,FEMA_5,FEMA_6,FEMA_7,FEMA_8,FEMA_9,...,g06,g13,g16,g17,g23g33g25,g29,g36,g46,g48,g51
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
MEDAGE_ACS,0.254,0.082,0.170,0.185,0.157,0.244,0.088,0.023,0.468,0.285,...,0.066,0.260,0.066,-0.037,0.376,0.158,0.312,0.210,0.125,0.113
BLACK_ACS,0.094,-0.238,0.062,0.187,-0.122,-0.060,-0.568,0.060,0.340,-0.526,...,-0.311,0.092,0.390,0.016,0.054,-0.023,0.042,0.462,0.438,-0.109
QNATAM_ACS,0.542,-0.046,0.574,-0.232,0.807,0.642,0.448,0.583,-0.076,0.399,...,-0.303,-0.533,0.283,0.532,0.437,0.442,-0.548,-0.067,-0.063,-0.301
QASIAN_ACS,0.210,-0.242,-0.123,0.072,0.073,0.021,0.120,-0.150,0.231,-0.000,...,-0.097,0.217,0.334,0.327,0.122,0.161,-0.165,0.513,0.228,0.199
QHISP_ACS,0.219,0.334,-0.119,0.634,0.461,0.389,0.377,0.396,0.516,0.275,...,0.271,0.369,0.253,-0.193,0.328,0.484,0.015,0.498,-0.027,0.560
QAGEDEP_ACS,0.500,0.291,0.171,0.379,0.375,0.617,0.562,0.329,0.512,0.381,...,0.234,0.148,0.116,0.053,0.561,0.435,0.598,0.139,0.183,0.269
QPUNIT_ACS,0.169,-0.072,-0.553,0.131,-0.407,0.058,-0.017,0.134,-0.187,0.136,...,-0.190,-0.370,-0.198,-0.430,0.363,-0.455,-0.355,-0.144,-0.372,-0.044
PRENTER_ACS,0.061,0.367,0.089,-0.130,0.304,-0.084,0.227,0.066,-0.167,-0.079,...,0.400,-0.114,0.457,0.191,-0.032,0.164,0.167,-0.040,0.233,0.102
QNRRES_ACS,0.461,0.330,-0.164,0.514,0.524,0.220,0.289,0.426,-0.290,0.859,...,0.661,0.441,0.166,-0.145,0.355,0.284,0.541,-0.281,0.330,0.161


### US Drop 1 Net Contribution

In [10]:
check_it("US_Drop1_NetContrib_raw.csv", rounder = 3)

QAGEDEP_ACS     27
QFEMALE_ACS     27
QSERV_ALT       27
QHISP_ACS       27
QFEMLBR         27
QNATAM_ACS      27
QESL_ALT        27
QSSBEN          27
QNOAUTO_ALT     27
QMOHO           27
QPOVTY          27
QNRRES_ACS      27
QFAM            27
QUNOCCHU_ACS    27
PERCAP_ALT      27
BLACK_ACS       27
MDGRENT_ALT     27
QRICH200K       27
MEDAGE_ACS      27
QFHH_ACS        27
PRENTER_ACS     27
POPDENS         27
QCVLUN          27
MHSEVAL_ALT     27
QED12LES_ALT    27
QEXTRCT_ALT     27
QASIAN_ACS      27
QPUNIT_ACS      27
dtype: int64

In [11]:
og.head()

,QAGEDEP_ACS,QFEMALE_ACS,QSERV_ALT,QHISP_ACS,QFEMLBR,QNATAM_ACS,QESL_ALT,QSSBEN,QNOAUTO_ALT,QMOHO,...,MEDAGE_ACS,QFHH_ACS,PRENTER_ACS,POPDENS,QCVLUN,MHSEVAL_ALT,QED12LES_ALT,QEXTRCT_ALT,QASIAN_ACS,QPUNIT_ACS
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
QAGEDEP_ACS,NaN,0.401,0.760,0.262,0.350,0.703,0.790,0.794,0.763,0.764,...,0.802,0.746,0.699,0.788,0.766,0.799,0.762,0.709,0.722,0.750
QFEMALE_ACS,-0.449,NaN,0.815,-0.187,-0.487,0.702,0.752,0.837,0.731,0.770,...,0.743,0.733,0.844,0.748,0.747,0.772,0.742,0.786,0.803,0.727
QSERV_ALT,0.745,0.749,NaN,0.727,0.646,0.447,0.682,0.468,0.524,0.693,...,0.669,0.632,0.453,0.614,0.666,0.502,0.693,0.412,0.440,0.643
QHISP_ACS,0.281,0.359,0.471,NaN,0.327,0.596,0.714,0.518,0.537,0.561,...,0.558,0.544,0.500,0.542,0.547,0.528,0.581,0.505,0.499,0.561
QFEMLBR,0.001,0.160,0.320,0.433,NaN,0.399,0.359,0.420,0.417,0.525,...,0.508,0.502,0.450,0.491,0.546,0.434,0.539,0.376,0.414,0.484


In [12]:
rpl.head()

,QAGEDEP_ACS,QFEMALE_ACS,QSERV_ALT,QHISP_ACS,QFEMLBR,QNATAM_ACS,QESL_ALT,QSSBEN,QNOAUTO_ALT,QMOHO,...,MEDAGE_ACS,QFHH_ACS,PRENTER_ACS,POPDENS,QCVLUN,MHSEVAL_ALT,QED12LES_ALT,QEXTRCT_ALT,QASIAN_ACS,QPUNIT_ACS
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
QAGEDEP_ACS,NaN,0.401,0.760,0.262,0.350,0.703,0.790,0.794,0.763,0.764,...,0.802,0.746,0.699,0.788,0.766,0.799,0.762,0.709,0.722,0.750
QFEMALE_ACS,-0.449,NaN,0.815,-0.187,-0.487,0.702,0.752,0.837,0.731,0.770,...,0.743,0.733,0.844,0.748,0.747,0.772,0.742,0.786,0.803,0.727
QSERV_ALT,0.745,0.749,NaN,0.727,0.646,0.447,0.682,0.468,0.524,0.693,...,0.669,0.632,0.453,0.614,0.666,0.502,0.693,0.412,0.440,0.643
QHISP_ACS,0.281,0.359,0.471,NaN,0.327,0.596,0.714,0.518,0.537,0.561,...,0.558,0.544,0.500,0.542,0.547,0.528,0.581,0.505,0.499,0.561
QFEMLBR,0.001,0.160,0.320,0.433,NaN,0.399,0.359,0.420,0.417,0.525,...,0.508,0.502,0.450,0.491,0.546,0.434,0.539,0.376,0.414,0.484


By inspection, there is one missing value in every column, so this is good!

### US Drop1 Net Contribution Ranks

In [13]:
check_it("US_Drop1_NetContrib_ranks.csv")

QAGEDEP_ACS     27
QFEMALE_ACS     27
QSERV_ALT       27
QHISP_ACS       27
QFEMLBR         27
QNATAM_ACS      27
QESL_ALT        27
QSSBEN          27
QNOAUTO_ALT     27
QMOHO           27
QPOVTY          27
QNRRES_ACS      27
QFAM            27
QUNOCCHU_ACS    27
PERCAP_ALT      27
BLACK_ACS       27
MDGRENT_ALT     27
QRICH200K       27
MEDAGE_ACS      27
QFHH_ACS        27
PRENTER_ACS     27
POPDENS         27
QCVLUN          27
MHSEVAL_ALT     27
QED12LES_ALT    27
QEXTRCT_ALT     27
QASIAN_ACS      27
QPUNIT_ACS      27
dtype: int64

In [14]:
og.head()

,QAGEDEP_ACS,QFEMALE_ACS,QSERV_ALT,QHISP_ACS,QFEMLBR,QNATAM_ACS,QESL_ALT,QSSBEN,QNOAUTO_ALT,QMOHO,...,MEDAGE_ACS,QFHH_ACS,PRENTER_ACS,POPDENS,QCVLUN,MHSEVAL_ALT,QED12LES_ALT,QEXTRCT_ALT,QASIAN_ACS,QPUNIT_ACS
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
QAGEDEP_ACS,NaN,5.0,2.0,8.0,8.0,1.0,1.0,2.0,1.0,2.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0
QFEMALE_ACS,5.0,NaN,1.0,14.0,4.0,2.0,2.0,1.0,2.0,1.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
QSERV_ALT,1.0,1.0,NaN,2.0,1.0,5.0,4.0,8.0,4.0,3.0,...,3.0,3.0,7.0,3.0,3.0,7.0,3.0,8.0,7.0,3.0
QHISP_ACS,11.0,6.0,5.0,NaN,10.0,3.0,3.0,6.0,3.0,4.0,...,4.0,4.0,5.0,4.0,4.0,6.0,4.0,5.0,5.0,4.0
QFEMLBR,27.0,17.0,9.0,5.0,NaN,7.0,9.0,9.0,7.0,5.0,...,5.0,5.0,8.0,8.0,5.0,9.0,5.0,10.0,9.0,5.0


In [15]:
rpl.head()

,QAGEDEP_ACS,QFEMALE_ACS,QSERV_ALT,QHISP_ACS,QFEMLBR,QNATAM_ACS,QESL_ALT,QSSBEN,QNOAUTO_ALT,QMOHO,...,MEDAGE_ACS,QFHH_ACS,PRENTER_ACS,POPDENS,QCVLUN,MHSEVAL_ALT,QED12LES_ALT,QEXTRCT_ALT,QASIAN_ACS,QPUNIT_ACS
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
QAGEDEP_ACS,NaN,5.0,2.0,8.0,8.0,1.0,1.0,2.0,1.0,2.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0
QFEMALE_ACS,5.0,NaN,1.0,14.0,4.0,2.0,2.0,1.0,2.0,1.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
QSERV_ALT,1.0,1.0,NaN,2.0,1.0,5.0,4.0,8.0,4.0,3.0,...,3.0,3.0,7.0,3.0,3.0,7.0,3.0,8.0,7.0,3.0
QHISP_ACS,11.0,6.0,5.0,NaN,10.0,3.0,3.0,6.0,3.0,4.0,...,4.0,4.0,5.0,4.0,4.0,6.0,4.0,5.0,5.0,4.0
QFEMLBR,27.0,17.0,9.0,5.0,NaN,7.0,9.0,9.0,7.0,5.0,...,5.0,5.0,8.0,8.0,5.0,9.0,5.0,10.0,9.0,5.0


Same as before -- looks good!

### State FEMA US Rank Correlations

***REPORTED IN SPIELMAN ET AL'S PAPER AS TABLE 2*** 

One value is actually reported 0.65 instead of 0.68 in their paper. All others match their paper.

In [64]:
check_it("state_fema_us_rank_correlations.csv", rounder = 14)

All values match!


### Drop 1 Place State Rank Change CA

In [17]:
check_it('drop1_place_state_rank_change_ca.csv')

All values match!


### Drop 1 Place State Quint Moves CA

I don't think they actually report this... do they? Maybe they just use it to make figures?

In [36]:
check_it('drop1_place_state_quint_moves_ca.csv')

r0    5
0     1
1     2
2     0
3     1
4     2
dtype: int64

In [19]:
og
# get rid of r0 column -- that's just row names

,r0,0,1,2,3,4
0,0,0.58,0.25,0.08,0.08,0.00
1,1,0.18,0.27,0.36,0.09,0.09
2,2,0.00,0.18,0.27,0.45,0.09
3,3,0.00,0.09,0.27,0.27,0.36
4,4,0.25,0.17,0.00,0.08,0.50


In [20]:
rpl

,r0,0,1,2,3,4
0,0,0.17,0.25,0.42,0.17,0.00
1,1,0.18,0.18,0.27,0.18,0.18
2,2,0.09,0.18,0.09,0.36,0.27
3,3,0.27,0.36,0.00,0.27,0.09
4,4,0.33,0.00,0.17,0.00,0.50


hmmmmm.... what is going on here?

### Drop 1 Place FEMA Rank Change FEMA 9

In [21]:
check_it('drop1_place_fema_rank_change_fema9.csv')

All_Counties    10
Drop_1           2
dtype: int64

In [22]:
rpl

,All_Counties,Drop_1
Unnamed: 0,,
1,Apache (1),Apache (1)
2,San Francisco (2),Navajo (4)
3,Gila (3),Imperial (5)
4,Navajo (4),Santa Cruz (9)
5,Imperial (5),San Francisco (2)
6,Modoc (6),Alpine (8)
7,La Paz (7),Gila (3)
8,Alpine (8),Tulare (16)
9,Santa Cruz (9),Modoc (6)


In [23]:
og

,All_Counties,Drop_1
Unnamed: 0,,
1,Apache (1),Apache (1)
2,San Francisco (2),Navajo (4)
3,Gila (3),San Francisco (2)
4,Navajo (4),Coconino (50)
5,Imperial (5),Santa Cruz (9)
6,Modoc (6),Imperial (5)
7,La Paz (7),Merced (20)
8,Alpine (8),Fresno (22)
9,Santa Cruz (9),Graham (39)


hmmm.... what is happening here??

### Drop 1 Place FEMA Quint Moves FEMA 9

In [24]:
check_it('drop1_place_fema_quint_moves_fema9.csv')

r0    5
0     1
1     0
2     1
3     1
4     1
dtype: int64

In [25]:
og

,r0,0,1,2,3,4
0,0,0.50,0.11,0.22,0.11,0.06
1,1,0.17,0.22,0.11,0.28,0.22
2,2,0.24,0.35,0.12,0.18,0.12
3,3,0.11,0.17,0.33,0.17,0.22
4,4,0.00,0.17,0.17,0.28,0.39


In [26]:
rpl

,r0,0,1,2,3,4
0,0,0.61,0.39,0.00,0.00,0.00
1,1,0.33,0.33,0.22,0.11,0.00
2,2,0.06,0.29,0.41,0.12,0.12
3,3,0.00,0.00,0.33,0.50,0.17
4,4,0.00,0.00,0.00,0.28,0.72


### Drop 1 Place USA Rank Change

In [27]:
check_it('drop1_place_usa_rank_change.csv')

All values match!


### Drop 1 Place USA Quint Moves

In [28]:
check_it('drop1_place_usa_quint_moves.csv')

All values match!


### Spearman State

In [59]:
check_it('spearman_state.csv')

g51_r          0
g48_r          0
g36_r          0
g06_r          0
g13_r          0
g16_r          0
g17_r          0
g29_r          0
g46_r          0
g23g33g25_r    0
g51_p          0
g48_p          0
g36_p          0
g06_p          0
g13_p          0
g16_p          0
g17_p          0
g29_p          0
g46_p          0
g23g33g25_p    0
dtype: int64

In [30]:
og

,g51_r,g48_r,g36_r,g06_r,g13_r,g16_r,g17_r,g29_r,g46_r,g23g33g25_r,g51_p,g48_p,g36_p,g06_p,g13_p,g16_p,g17_p,g29_p,g46_p,g23g33g25_p
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
Region,-0.019988,-0.074492,-0.094659,-0.192470,-0.182067,-0.481748,0.061786,0.202351,-0.068949,0.249719,0.818690,0.236813,0.464277,0.147765,0.021625,0.000930,0.537293,0.030101,0.582253,0.120181
US,-0.302719,0.237168,-0.204905,0.116491,-0.236712,-0.511628,-0.093180,0.249665,0.327878,-0.093433,0.000378,0.000136,0.110133,0.383850,0.002664,0.000386,0.351593,0.007127,0.007198,0.566345


In [31]:
rpl

,g51_r,g48_r,g36_r,g06_r,g13_r,g16_r,g17_r,g29_r,g46_r,g23g33g25_r,g51_p,g48_p,g36_p,g06_p,g13_p,g16_p,g17_p,g29_p,g46_p,g23g33g25_p
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
Region,0.896188,0.680187,0.606910,0.690978,0.796931,0.880197,0.336492,0.816752,0.866486,0.935272,1.995680e-48,7.553754e-36,1.702705e-07,1.932681e-09,3.396767e-36,3.518791e-15,5.443364e-04,9.383998e-29,5.583511e-21,9.898190e-19
US,0.675348,0.615105,0.787716,0.531330,0.503329,0.661311,0.504419,0.902691,0.605427,0.745591,3.565129e-19,7.861542e-28,3.068729e-14,1.775837e-05,1.361097e-11,1.024956e-06,6.464506e-08,3.412373e-43,7.233126e-08,3.391702e-08


oooof check this!

### County in Region Rank

In [32]:
check_it('County_in_Region_Rank.csv')

GEOID                    3109
fema_region_sovi_rank    3109
us_sovi_rank             3109
dtype: int64

In [33]:
og.loc[~test["GEOID"]].head()

,GEOID,fema_region_sovi_rank,us_sovi_rank
67,g02013,NaN,NaN
68,g02016,NaN,NaN
69,g02020,NaN,NaN
70,g02050,NaN,NaN
71,g02060,NaN,NaN


In [34]:
rpl.loc[~test["GEOID"]].head()

,GEOID,fema_region_sovi_rank,us_sovi_rank
Unnamed: 0,,,
67,NaN,NaN,NaN
68,NaN,NaN,NaN
69,NaN,NaN,NaN
70,NaN,NaN,NaN
71,NaN,NaN,NaN


Looks like the issue is Hawaii and Alaska again -- they were excluded from the analysis. I haven't found mention of this choice in their paper, but presumably they chose to exclude HI and AK from their FEMA region analysis because contiguous regions are more similar regarding their susceptability to natural hazards.

### Spearman Region

In [62]:
check_it('spearman_region.csv', rounder = 12)

All values match!


At this point, I have checked all results generated by my scripts. There are a couple of files I haven't generated yet, perhaps they are created in the script I haven't looked at yet.

Issues:
- Spearman state
- CA and FEMA9 Quint Moves/Rank Change